In [45]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:90%;}
div.CodeMirror {font-family:Consolas; font-size:18pt;}
div.output {font-size:19pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:19pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
span.toc-item-num{display:none;}
div.text_cell_render ul li{font-size:16pt;padding:5px;}
div.CodeMirror-lines > div {padding-left:10px;}
table.dataframe{font-size:19px;}
</style>
"""))

전국 평균 평당 분양가격(2013년 9월부터 2015년 8월까지).csv (지역명, 평당분양가격, 연도, 월)
- 지역명2 : 지역명필드는 라벨인코딩하여 추가
- 평당분양가격n : 평당분양가격의 normalization 스케일 조정하여 추가
- 평당분양가격s : 평당분양가격의 standardization 스케일 조정하여 추가
- 연도n : 연도의 normalization 스케일 조정하여 추가
- 연도s : 연도의 standardization 스케일 조정하여 추가
- 월n   : 월의 normalization 스케일 조정하여 추가
- 월s   : 월의 standardization 스케일 조정하여 추가

## 1. 데이터 가져오기

In [7]:
import pandas as pd
csvdata = pd.read_csv(r'D:\ai\Downloads\shareData\전국 평균 평당 분양가격(2013년 9월부터 2015년 8월까지).csv',
                encoding='cp949')
df = csvdata.melt(id_vars="지역", var_name="연도월", value_name="평당분양가격")
df["연도"] = df["연도월"].apply(lambda x : int(x.split("년")[0]))
df["월"] = df["연도월"].apply(lambda x : int(x.split("년")[-1][:-1]))
df.drop('연도월', axis=1, inplace=True)
df = df.rename(columns={'지역':'지역명'})
df.head(2)

,지역명,평당분양가격,연도,월
0,서울,18189,2013,12
1,부산,8111,2013,12


## 2. 지역명의 라벨 인코딩

In [8]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
le = LabelEncoder()
df['지역명2'] = le.fit_transform(df['지역명'])
df.head()

,지역명,평당분양가격,연도,월,지역명2
0,서울,18189,2013,12,8
1,부산,8111,2013,12,7
2,대구,8080,2013,12,5
3,인천,10204,2013,12,11
4,광주,6098,2013,12,4


# 3. normalization 스케일 조정

In [33]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np
# 열별 스케일 조정
year_scaler = MinMaxScaler()
# year_scaler.fit(np.array(df['연도']).reshape(-1,1))
# df['연도n'] = year_scaler.transform(np.array(df['연도']).reshape(-1,1))
df['연도n'] = year_scaler.fit_transform(df[['연도']].values)

mon_scaler = MinMaxScaler()
df['월n'] = mon_scaler.fit_transform(df[['월']])

price_scaler = MinMaxScaler()
df['평당분양가격n'] = price_scaler.fit_transform(df[['평당분양가격']])

df.loc[::100]
# 2025년 3월에 경남(2번지역) (2, year_scaler.transform(2025), mon_scaler.transform(3))

,지역명,평당분양가격,연도,월,지역명2,연도n,월n,평당분양가격n,연도s,월s
0,서울,18189,2013,12,8,0.0,1.000000,0.835270,-2.366432,1.767475
100,경남,6610,2014,5,2,0.5,0.363636,0.066463,-0.591608,-0.294579
200,전남,5768,2014,11,12,0.5,0.909091,0.010557,-0.591608,1.472896
300,충남,6975,2015,5,15,1.0,0.363636,0.090698,1.183216,-0.294579


In [27]:
# 변수들 전체 스케일 조정
from sklearn.preprocessing import MinMaxScaler
import numpy as np
scaler = MinMaxScaler()
df[['연도n', '월n', '평당분양가격n']] = scaler.fit_transform(df[['연도', '월', '평당분양가격']])
df.head()

,지역명,평당분양가격,연도,월,지역명2,연도n,월n,평당분양가격n,연도s,월s
0,서울,18189,2013,12,8,0.0,1.0,0.835270,-2.366432,1.767475
1,부산,8111,2013,12,7,0.0,1.0,0.166124,-2.366432,1.767475
2,대구,8080,2013,12,5,0.0,1.0,0.164066,-2.366432,1.767475
3,인천,10204,2013,12,11,0.0,1.0,0.305093,-2.366432,1.767475
4,광주,6098,2013,12,4,0.0,1.0,0.032468,-2.366432,1.767475


In [13]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np
# 열별 스케일 조정
year_scaler = MinMaxScaler()
year_scaler.fit(df['연도'].values.reshape(-1, 1))
df['연도n']= year_scaler.transform(df['연도'].values.reshape(-1,1))

mon_scaler = MinMaxScaler()
mon_scaler.fit( df['월'].values.reshape(-1,1) )
df['월n']= mon_scaler.transform( df['월'].values.reshape(-1,1) )

price_scaler = MinMaxScaler()
price_scaler.fit( df['평당분양가격'].values.reshape(-1,1) )
df['평당분양가격n']= price_scaler.transform(df['평당분양가격'].values.reshape(-1,1))

df.loc[::100]

,지역명,평당분양가격,연도,월,지역명2,연도n,월n,평당분양가격n,연도s,월s
0,서울,18189,2013,12,8,0.0,1.000000,0.835270,-2.366432,1.767475
100,경남,6610,2014,5,2,0.5,0.363636,0.066463,-0.591608,-0.294579
200,전남,5768,2014,11,12,0.5,0.909091,0.010557,-0.591608,1.472896
300,충남,6975,2015,5,15,1.0,0.363636,0.090698,1.183216,-0.294579


# standardization 

In [14]:
from sklearn.preprocessing import StandardScaler
import numpy as np
year_scaler = StandardScaler()
year_scaler.fit(np.array(df['연도']).reshape(-1,1))
df['연도s']= year_scaler.transform(np.array(df['연도']).reshape(-1,1))

mon_scaler = StandardScaler()
mon_scaler.fit( np.array(df['월']).reshape(-1,1) )
df['월s']=mon_scaler.transform(np.array(df['월']).reshape(-1,1))

price_scaler = StandardScaler()
price_scaler.fit( np.array(df['평당분양가격']).reshape(-1,1) )
df['평당분양가격s']= price_scaler.transform(df['평당분양가격']).reshape(-1,1)

df = df[['지역명','지역명2',
    '평당분양가격','평당분양가격n','평당분양가격s',
    '연도','연도n','연도s','월','월n','월s']]
df.loc[::100]

ValueError: Expected 2D array, got 1D array instead:
array=[18189.  8111.  8080. 10204.  6098.  8321.  8090. 10855.  7601.  6230.
  6589.  6365.  6282.  5678.  6168.  6473.  7674. 17925.  8111.  8080.
 10204.  7326.  8321.  8090. 10855.  7600.  6230.  6589.  6365.  6281.
  5678.  6168.  6485.  7900. 17925.  9078.  8077. 10408.  7611.  8321.
  8090. 10791.  7532.  6230.  6611.  6379.  5946.  5678.  6234.  6502.
  7900. 18016.  8965.  8101. 10408.  7346.  8341.  8153. 10784.  7814.
  6141.  6625.  6287.  5966.  5696.  6317.  6610.  7900. 18098.  9402.
  8267. 10000.  7346.  8341.  8153. 10876.  7908.  6373.  6678.  6552.
  6277.  5736.  6412.  6599.  7900. 19446.  9501.  8274.  9844.  7523.
  8341.  8153. 10646.  7934.  6350.  6598.  6591.  6306.  5656.  6409.
  6610.  7900. 18867.  9453.  8360. 10058.  7659.  8333.  8153. 10266.
  8067.  6350.  6587.  6644.  6351.  5609.  6554.  6615.  7914. 18742.
  9457.  8360.  9974.  7612.  8333.  8153. 10124.  8067.  6268.  6586.
  6805.  6319.  57

In [86]:
df.shape

(357, 11)